# Projet SIEM : Suricata, Filebeat, Elasticsearch

## Objectifs

Ce projet vous permettra de :
- Vérifier le bon fonctionnement de la stack SIEM
- Analyser les données indexées par Filebeat (Suricata)
- Détecter des anomalies sans ML
- Détecter des anomalies avec ML (Isolation Forest)

## Modalité
- groupe de 3 à 4
- output : projet git avec ce notebook détaillé et complété

## Architecture SIEM

```
Suricata   →   Filebeat   →   Elasticsearch   →   Kibana
(IDS/HIDS)   (Collecteur)      (Stockage)   (Visualisation/Alerting)
```

## Prérequis

1. Démarrer la stack :
```bash
docker-compose -f docker-compose-siem.yml up -d
```

2. Attendre quelques minutes que Suricata génère des logs et que Filebeat les indexe dans Elasticsearch.

In [ ]:
# Configuration et connexion à Elasticsearch
from elasticsearch import Elasticsearch
from datetime import datetime, timedelta
import json
import subprocess
import os
import warnings
from urllib3.exceptions import InsecureRequestWarning

warnings.simplefilter("ignore", InsecureRequestWarning)
# Configuration
ES_HOST = "https://localhost:9200"
ES_USER = "elastic"
ES_PASSWORD = "changeme"  # Modifiez selon votre .env

# Connexion
es = Elasticsearch(
    [ES_HOST],
    basic_auth=(ES_USER, ES_PASSWORD),
    verify_certs=False
)

# Vérification de la connexion
health = es.cluster.health()
print(f"✅ Cluster Elasticsearch: {health['status']} ({health['number_of_nodes']} nœuds)")

## 1. Vérification de la stack

In [ ]:
# Vérification des services Docker
services = ['es01', 'es02', 'es03', 'kibana', 'suricata', 'filebeat']
running = []

for service in services:
    try:
        result = subprocess.run(
            ['docker', 'ps', '--filter', f'name={service}', '--format', '{{.Names}}'],
            capture_output=True, text=True, timeout=5
        )
        if service in result.stdout:
            running.append(service)
            print(f"✅ {service}")
        else:
            print(f"❌ {service}")
    except:
        print(f"❌ {service}")

if len(running) == len(services):
    print(f"\n✅ Tous les services sont démarrés ({len(running)}/{len(services)})")
else:
    print(f"\n⚠️  Services démarrés: {len(running)}/{len(services)}")

## 2. Vérification de l'injection des données

In [ ]:
# Recherche des index Suricata
def get_suricata_index():
    """Retourne le nom de l'index Suricata le plus récent"""
    try:
        indices = es.indices.get(index="suricata-*")
        if indices:
            return sorted(indices.keys())[-1]
    except:
        pass
    return "suricata-*"

index_name = get_suricata_index()

# Comptage des documents
try:
    count = es.count(index=index_name)
    print(f"📊 Index: {index_name}")
    print(f"📈 Nombre de documents: {count['count']:,}")
    
    # Exemple de document
    if count['count'] > 0:
        sample = es.search(index=index_name, size=1, query={"match_all": {}})
        if sample['hits']['hits']:
            doc = sample['hits']['hits'][0]['_source']
            print(f"\n📄 Exemple de document:")
            print(f"   Type: {doc.get('event_type', 'N/A')}")
            print(f"   Timestamp: {doc.get('@timestamp', doc.get('timestamp', 'N/A'))}")
            if 'src_ip' in doc:
                print(f"   Source: {doc.get('src_ip')}:{doc.get('src_port', 'N/A')}")
                print(f"   Destination: {doc.get('dest_ip')}:{doc.get('dest_port', 'N/A')}")
            if 'alert' in doc:
                alert = doc['alert']
                print(f"   Alerte: {alert.get('signature', 'N/A')}")
                print(f"   Sévérité: {alert.get('severity', 'N/A')}")
except Exception as e:
    print(f"❌ Erreur: {e}")

## 4. Simuler des comportements anormaux


#### Simulation de scans suspects (en ligne de commande ou en python) 

In [ ]:
 # TODO

#### Simulation de burst HTTP en ligne de commande (en ligne de commande ou en python) 

In [ ]:
# TODO

#### Simulation de [à continuer]

## 4. Détection d'anomalies sans ML

## 4. Détection d'anomalie (sans Machine Learning)

L'objectif est de construire des règles qui détectent vos simulations de comportements précédents.

Un exemple de détection d'anomalies basiques basées sur des règles:
```python
def detect_anomalies_basic():
    """Détecte des anomalies sans ML"""
    anomalies = []
    
    # 1. IPs sources avec beaucoup d'alertes différentes (scan suspect)
    query = {
        "size": 0,
        "query": {"term": {"event_type": "alert"}},
        "aggs": {
            "suspicious_ips": {
                "terms": {"field": "src_ip", "size": 10},
                "aggs": {
                    "unique_signatures": {"cardinality": {"field": "alert.signature_id"}},
                    "unique_dest_ips": {"cardinality": {"field": "dest_ip"}},
                    "total_alerts": {"value_count": {"field": "event_type"}}
                }
            }
        }
    }
    
    result = es.search(index=index_name, body=query)
    
    print("Détection d'anomalies (règles basiques)\n")

    for bucket in result['aggregations']['suspicious_ips']['buckets']:
        ip = bucket['key']
        unique_sigs = bucket['unique_signatures']['value']
        unique_dests = bucket['unique_dest_ips']['value']
        total = bucket['total_alerts']['value']
        
        # Critères d'anomalie
        if unique_sigs > 3 or unique_dests > 5:
            anomalies.append({
                "ip": ip,
                "type": "Scan suspect",
                "signatures": unique_sigs,
                "destinations": unique_dests,
                "total": total
            })
            print(f"🚨 {ip}: {unique_sigs} signatures, {unique_dests} destinations ({total} alertes)")

    return anomalies

anomalies = detect_anomalies_basic()
if not anomalies:
    print("\nAucune anomalie détectée avec les règles basiques")
```

In [ ]:
# TODO

## 5. Détection d'anomalies avec ML

L'objectif est d'à partir de données labelisées comme étant anomarles ou non, construire un jeu de donnée et entrainer un algorithme de machine learning (classifier binaire) sur ces données. Le modèle devra ensuite etre appelé pour prédire les futurs comportements anormaux.

Indice: le modèle Isolation Forest pourrait etre utile

In [43]:
# TODO

## Bonus
- améliorer la stack (ex: ajout de Wazuh)
- dashboard Kibana pour voir en live les simulations de comportements anormaux et les détection d'anomalie (timeline des alertes, etc.)
- analyse statistique avancée
- simulations de comportement anormaux avancée
- utilisation du module de détection d'anomalie d'Elasticsearch (https://www.elastic.co/docs/explore-analyze/machine-learning/anomaly-detection)
- collaboration en groupe sur le projet Git (Pull requests, commits, etc.)
- utilisation de docker / docker compose / devcontainer
- etc.